# Section 1: Predicting Ratings with Sklearn's NMF

To keep the computation reasonable, I decided to work with a smaller sample of the data. I randomly selected 500 users and 500 movies from the training set. Using this subset, I built a user-item matrix where each entry represents a user's rating for a movie. Since the matrix needs to be fully filled for sklearn's NMF to work, I replaced all missing values with zeros.

Next, I applied the non-negative matrix factorization algorithm provided by the sklearn library. I set the number of components to 20. The model learns two sets of latent features: one for users and one for movies. When these two matrices are multiplied together, they produce a reconstructed matrix of predicted ratings.

After training the model, I tested its performance by predicting ratings in the test set. I only evaluated ratings for users and movies that appeared in the subsampled training data. I then compared these predicted ratings with the actual ones from the test set and calculated the root mean squared error. The RMSE turned out to be approximately 3.30.

Since ratings range from 1 to 5, an RMSE this high indicates that the model struggled to make accurate predictions. I’ll explain why this happened and explore possible improvements in the next section.


In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error

movie_data = pd.read_csv("movies.csv")
user_data = pd.read_csv("users.csv")
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

selected_users = train_data['uID'].drop_duplicates().sample(500, random_state=42)
selected_movies = train_data['mID'].drop_duplicates().sample(500, random_state=42)

subset = train_data[
    (train_data['uID'].isin(selected_users)) & (train_data['mID'].isin(selected_movies))
]

u_index = subset['uID'].unique()
m_index = subset['mID'].unique()

u_map = {u: i for i, u in enumerate(u_index)}
m_map = {m: j for j, m in enumerate(m_index)}

matrix = np.zeros((len(u_index), len(m_index)))
for row in subset.itertuples(index=False):
    matrix[u_map[row.uID], m_map[row.mID]] = row.rating

model = NMF(n_components=20, init='random', random_state=42, max_iter=200)
user_factors = model.fit_transform(matrix)
movie_factors = model.components_
reconstructed = np.dot(user_factors, movie_factors)

eval_set = test_data[
    (test_data['uID'].isin(u_index)) & (test_data['mID'].isin(m_index))
]

actual, predicted = [], []
for row in eval_set.itertuples(index=False):
    actual.append(row.rating)
    predicted.append(reconstructed[u_map[row.uID], m_map[row.mID]])

score = np.sqrt(mean_squared_error(actual, predicted))
print(f"RMSE: {score:.4f}")


C:\Users\tejas\miniconda3\envs\col\Lib\site-packages\sklearn\decomposition\_nmf.py:1728: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


RMSE: 3.2955


# Section 2: Discussion – Limitations of Sklearn’s NMF

The model achieved an RMSE of around 3.30, which is fairly high considering that the ratings range from 1 to 5. This suggests that the predictions were not very accurate. One major reason for this is how sparse the original dataset is. Since sklearn’s NMF cannot handle missing entries directly, all the missing ratings had to be filled in with zeros. That approach introduces a problem: it can make the model interpret unrated items as negative feedback, even though no rating simply means no information.

Another challenge is that sklearn’s implementation doesn’t include any special handling for missing data. Libraries designed specifically for recommendation systems, like Surprise or Implicit, often have ways of dealing with this. In contrast, sklearn assumes the matrix is complete, which doesn’t match the way real-world rating data is structured.

The model also lacks flexibility when it comes to capturing bias. In many cases, users have consistent tendencies to rate higher or lower than average, and some movies tend to receive higher ratings overall. Sklearn’s NMF doesn’t account for this, so it can miss important patterns in the data. Additionally, because we were working with a subset of users and movies, the model may not have had enough data to properly learn meaningful features. With more data, it could also be at risk of overfitting if regularization isn’t properly handled.

Another common issue in recommendation systems is the cold-start problem, where the model struggles to make predictions for users or items it hasn’t seen before. NMF in sklearn isn’t designed to deal with this situation.

To improve results, one approach would be to use a more specialized library that handles sparse input better and includes built-in tools for bias correction and regularization. It might also help to avoid filling missing values with zeros. Instead, using averages based on users or items, or employing imputation techniques, could provide better input to the model. A hybrid method that mixes matrix factorization with similarity-based techniques could also capture relationships more effectively.

Overall, sklearn’s NMF is helpful for exploring the basic idea of matrix factorization, but it’s not ideal for building a practical recommendation system. More specialized tools are better suited to handle the real-world complexities of user-item rating data.
